# Annotations Implementation

## Setup

In [ ]:
import sys
import os
main_dir = os.path.dirname(os.path.dirname(os.getcwd()))
sys.path.append(main_dir)

import argparse
import os
import time

import numpy as np

from RemoteSensing.changedetection.configs.config import get_config

import torch
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import DataLoader
from tqdm import tqdm
from RemoteSensing.changedetection.datasets.make_data_loader import SemanticChangeDetectionDatset, make_data_loader
from RemoteSensing.changedetection.utils_func.metrics import Evaluator
from RemoteSensing.changedetection.models.STMambaSCD import STMambaSCD
import RemoteSensing.changedetection.utils_func.lovasz_loss as L
from torch.optim.lr_scheduler import StepLR
from dotenv import load_dotenv

from RemoteSensing.changedetection.utils_func.mcd_utils import accuracy, SCDD_eval_all, AverageMeter

In [ ]:
load_dotenv()
def getPath(env_path):
    return os.path.expanduser(os.getenv(env_path))

VSSM_MODEL_PATH = getPath('VSSMBASEPATH')

SECOND_DATASET_PATH = getPath('SECONDDATASETPATH')

SECOND_TRAIN_DATASET_PATH = os.path.join(SECOND_DATASET_PATH, 'train')
SECOND_TEST_DATASET_PATH = os.path.join(SECOND_DATASET_PATH, 'test')
SECOND_TRAIN_DATA_LIST_PATH = os.path.join(SECOND_DATASET_PATH, 'train.txt')
SECOND_TEST_DATA_LIST_PATH = os.path.join(SECOND_DATASET_PATH, 'test.txt')

print(SECOND_DATASET_PATH)

In [ ]:
configs_path = os.path.join(main_dir, 'RemoteSensing/changedetection/configs/vssm1/vssm_base_224.yaml')
model_path = os.path.join(main_dir, 'RemoteSensing/saved_models')

MODEL_PATH = os.path.join(
    main_dir, "RemoteSensing/saved_models/CA_spatial_2/32000_model_0.244.pth"
)

train_data_list = []
with open(SECOND_TRAIN_DATA_LIST_PATH, 'r') as f:
    for line in f:
        train_data_list.append(line.strip())

test_data_list = []
with open(SECOND_TEST_DATA_LIST_PATH, 'r') as f:
    for line in f:
        test_data_list.append(line.strip())

class ARGS:
    def __init__(self):
        self.cfg = configs_path
        self.opts = None
        self.pretrained_weight_path = VSSM_MODEL_PATH
        self.dataset = 'SECOND'
        self.type = 'train'
        self.train_dataset_path = SECOND_TRAIN_DATASET_PATH

        self.test_dataset_path = SECOND_TEST_DATASET_PATH

        
        self.shuffle = True
        self.batch_size = 4
        self.crop_size = 256
        self.train_data_name_list = train_data_list
        self.test_data_name_list = test_data_list
        self.start_iter = 0
        self.cuda = True
        self.max_iters = 800000
        self.model_type = 'MambaSCD_Base'
        self.model_param_path = model_path
        self.resume = MODEL_PATH
        self.learning_rate = 1e-4
        self.momentum = 0.9
        self.weight_decay = 5e-4

In [ ]:
args = ARGS()
config = get_config(args)

train_data_loader = make_data_loader(args)

deep_model = STMambaSCD(
    output_cd = 2, 
    output_clf = 7,
    pretrained=args.pretrained_weight_path,
    patch_size=config.MODEL.VSSM.PATCH_SIZE, 
    in_chans=config.MODEL.VSSM.IN_CHANS, 
    num_classes=config.MODEL.NUM_CLASSES, 
    depths=config.MODEL.VSSM.DEPTHS, 
    dims=config.MODEL.VSSM.EMBED_DIM, 
    # ===================
    ssm_d_state=config.MODEL.VSSM.SSM_D_STATE,
    ssm_ratio=config.MODEL.VSSM.SSM_RATIO,
    ssm_rank_ratio=config.MODEL.VSSM.SSM_RANK_RATIO,
    ssm_dt_rank=("auto" if config.MODEL.VSSM.SSM_DT_RANK == "auto" else int(config.MODEL.VSSM.SSM_DT_RANK)),
    ssm_act_layer=config.MODEL.VSSM.SSM_ACT_LAYER,
    ssm_conv=config.MODEL.VSSM.SSM_CONV,
    ssm_conv_bias=config.MODEL.VSSM.SSM_CONV_BIAS,
    ssm_drop_rate=config.MODEL.VSSM.SSM_DROP_RATE,
    ssm_init=config.MODEL.VSSM.SSM_INIT,
    forward_type=config.MODEL.VSSM.SSM_FORWARDTYPE,
    # ===================
    mlp_ratio=config.MODEL.VSSM.MLP_RATIO,
    mlp_act_layer=config.MODEL.VSSM.MLP_ACT_LAYER,
    mlp_drop_rate=config.MODEL.VSSM.MLP_DROP_RATE,
    # ===================
    drop_path_rate=config.MODEL.DROP_PATH_RATE,
    patch_norm=config.MODEL.VSSM.PATCH_NORM,
    norm_layer=config.MODEL.VSSM.NORM_LAYER,
    downsample_version=config.MODEL.VSSM.DOWNSAMPLE,
    patchembed_version=config.MODEL.VSSM.PATCHEMBED,
    gmlp=config.MODEL.VSSM.GMLP,
    use_checkpoint=config.TRAIN.USE_CHECKPOINT,
    ) 

deep_model.load_state_dict(torch.load(MODEL_PATH))
deep_model = deep_model.cuda()
deep_model.eval() 

=> merge config from /home/buddhiw/ChangeDetection/CDMamba/RemoteSensing/changedetection/configs/vssm1/vssm_base_224.yaml
Successfully load ckpt /home/buddhiw/ChangeDetection/VSSModels/pretrained/vssm_base_0229_ckpt_epoch_237.pth
_IncompatibleKeys(missing_keys=['outnorm0.weight', 'outnorm0.bias', 'outnorm1.weight', 'outnorm1.bias', 'outnorm2.weight', 'outnorm2.bias', 'outnorm3.weight', 'outnorm3.bias'], unexpected_keys=['classifier.norm.weight', 'classifier.norm.bias', 'classifier.head.weight', 'classifier.head.bias'])
False


/home/buddhiw/miniconda3/envs/Mamba/lib/python3.13/site-packages/torch/nn/init.py:511: UserWarning: Initializing zero-element tensors is a no-op
  warnings.warn("Initializing zero-element tensors is a no-op")


<All keys matched successfully>

# Ground Truth Annotations

In [ ]:
IMAGE_ID = "06350"

label_A = os.path.join(SECOND_TRAIN_DATASET_PATH, "GT_T1", f"{IMAGE_ID}.png")
label_B = os.path.join(SECOND_TRAIN_DATASET_PATH, "GT_T2", f"{IMAGE_ID}.png")
label_cd = os.path.join(SECOND_TRAIN_DATASET_PATH, "GT_CD", f"{IMAGE_ID}.png")

